In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
"""
Purpose: To test the neuron decomposition pipeline using 
the vdi for microns that accesses the data through the caveclient
"""

'\nPurpose: To test the neuron decomposition pipeline using \nthe vdi for microns that accesses the data through the caveclient\n'

# Setting up the Cave Client

In [3]:
from caveclient import CAVEclient
client = CAVEclient()
client.auth.get_new_token()

New Tokens need to be acquired by hand. Please follow the following steps:
                1) Go to: https://global.daf-apis.com/auth/api/v1/create_token to create a new token.
                2) Log in with your Google credentials and copy the token shown afterward.
                3a) Save it to your computer with: client.auth.save_token(token="PASTE_YOUR_TOKEN_HERE")
                or
                3b) Set it for the current session only with client.auth.token = "PASTE_YOUR_TOKEN_HERE"
                Note: If you need to save or load multiple tokens, please read the documentation for details.
                Warning! Creating a new token by finishing step 2 will invalidate the previous token!


In [4]:
CAVE_TOKEN  = "INSERT TOKEN HERE"

#this token is an example token but is not valid
CAVE_TOKEN="ba0a1b546e9b87f402c5d8c1d1d45bca" 

In [5]:
# Saving the token to caveclient package and as a local env file
try:
    client.auth.save_token(token=CAVE_TOKEN)
except Exception as e:
    print(e)

filepath ="./.env" 
with open(filepath,"w+") as f:
    f.write(f"CAVE_TOKEN={CAVE_TOKEN}")

Key "token" already exists in token file "/root/.cloudvolume/secrets/global.daf-apis.com-cave-secret.json"


# Importing the VDI with microns_cave interface


In [6]:
from neurd.cave_interface import CaveInterface
from pathlib import Path

client = CaveInterface(
    "minnie65_public",
    env_filepath=Path(filepath).absolute()
)
client

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/usr/local/lib/python3.8/dist-packages/python_jsonschema_objects/__init__.py:113: UserWarning: Schema id not specified. Defaulting to 'self'
  warnings.warn("Schema id not specified. Defaulting to 'self'")


# Accessing the data with the Cave Interface (we are just checking to make sure cave client access is working, lines of code not necessary for VDI to function)

In [7]:
nucleus_seg_ids = client.segment_ids_with_nucleus()
nucleus_seg_ids[:10]

Table Owner Notice on nucleus_ref_neuron_svm: Please cite https://doi.org/10.1101/2022.07.20.499976 when using this table.


array([                 0, 864691131906133483, 864691132144578807,
       864691132180274289, 864691132187700262, 864691132255729245,
       864691132276185673, 864691132350062180, 864691132405111073,
       864691132412868837])

In [8]:
# finding a segement id from the nuclues id
nucleus_id = 197805

nuc_df = client.neuron_nucleus_df()
seg_id = nuc_df.query(f"id_ref=={nucleus_id}")[
    "pt_root_id"].to_list()[0]
seg_id

Table Owner Notice on nucleus_ref_neuron_svm: Please cite https://doi.org/10.1101/2022.07.20.499976 when using this table.


864691135502491741

In [9]:
# downloading the mesh
mesh = client.mesh_from_seg_id(
    seg_id
)

In [10]:
# downloading the synapses
syn_df = client.pre_post_df_from_seg_id(seg_id)
syn_df

,id,created,superceded_id,valid,size,pre_pt_supervoxel_id,pre_pt_root_id,post_pt_supervoxel_id,post_pt_root_id,pre_pt_position,post_pt_position,ctr_pt_position
0,60791196,2020-11-04 06:58:29.963477+00:00,NaN,t,8540,81295211040549893,864691135502491741,81295211040561026,864691135446598290,"[119948, 204892, 25222]","[120012, 204940, 25228]","[119972, 204962, 25221]"
1,55276068,2020-11-04 13:26:20.169296+00:00,NaN,t,1480,80521704610504328,864691135502491741,80521704610511535,864691135542422980,"[114594, 208994, 25324]","[114524, 208936, 25324]","[114538, 208976, 25325]"
2,77044514,2020-11-04 07:03:29.958267+00:00,NaN,t,1884,82835626965138956,864691135502491741,82835626965146533,864691135498532371,"[131206, 147428, 26140]","[131232, 147418, 26148]","[131210, 147454, 26145]"
3,99612983,2020-11-04 07:03:23.818013+00:00,NaN,t,816,84178611632143296,864691135502491741,84178611632147211,864691133673798118,"[140958, 192060, 25867]","[140918, 192048, 25869]","[140944, 192064, 25870]"
4,92346441,2020-11-04 07:15:03.970582+00:00,NaN,t,25688,83477054561109525,864691135502491741,83477054561117875,864691136176212230,"[136004, 207680, 26253]","[135976, 207712, 26257]","[135980, 207692, 26255]"
...,...,...,...,...,...,...,...,...,...,...,...,...
2258,92314208,2020-11-04 10:44:18.103393+00:00,NaN,t,6108,83473893398472858,864691135658301314,83473824678991778,864691135502491741,"[136222, 184010, 26102]","[136212, 183874, 26109]","[136259, 183982, 26102]"
2259,89540754,2020-11-04 11:13:37.958065+00:00,NaN,t,9260,83685068283331835,864691136105933529,83685068283341685,864691135502491741,"[137574, 184638, 25563]","[137526, 184604, 25576]","[137552, 184610, 25572]"
2260,104897868,2020-11-04 14:39:15.101242+00:00,NaN,t,11104,84741355561256766,864691136617619675,84741355561261515,864691135502491741,"[145044, 190290, 26791]","[145074, 190372, 26796]","[145084, 190310, 26798]"
2261,113438365,2020-11-04 11:06:17.563648+00:00,NaN,t,2780,85163636544799639,864691135762144822,85163636544804747,864691135502491741,"[148390, 190994, 25583]","[148408, 191006, 25588]","[148436, 190943, 25588]"


# Using the VDI Microns Cave

In [ ]:
# assumes your caveclient token is stored in an env file
from neurd.vdi_microns_cave import volume_data_interface as vdi

In [ ]:
vdi.client.segment_ids_with_nucleus()

In [ ]:
vdi.voxel_to_nm_scaling

# Neuron Preprocessing Pipeline

In [ ]:
from mesh_tools import trimesh_utils as tu
from datasci_tools import ipyvolume_utils as ipvu
from neurd import neuron_visualizations as nviz

In [ ]:
segment_id = 864691135502491741

# Step 0: Loading Mesh

In [ ]:
mesh = vdi.fetch_segment_id_mesh(
    segment_id,
    plot = False
)
mesh

In [ ]:
from datasci_tools import pipeline

products = pipeline.PipelineProducts()
products

# Step 0.1 (optional): Visualizing Synapses with Mesh

In [ ]:
syn_dict = vdi.segment_id_to_synapse_dict(segment_id)

In [ ]:
from datasci_tools import ipyvolume_utils as ipvu
ipvu.plot_objects(
    mesh,
    scatters = [
        syn_dict["postsyn"]["synapse_coordinates"],
        syn_dict["presyn"]["synapse_coordinates"]
    ],
    scatters_colors=["blue","yellow"]
)

# Step 1: Decimation

In [ ]:
decimation_parameters = dict(
    decimation_ratio =0.25,
)

In [ ]:
mesh_decimated = tu.decimate(
    mesh,
    **decimation_parameters
)

In [ ]:
ipvu.plot_objects(mesh_decimated)

In [ ]:
# saving any parameters used or output products for later use
products.set_stage_attrs(
    stage = "decimation",
    attr_dict = dict(
        decimation_parameters = decimation_parameters,
        segment_id = segment_id,
    ),
    
)

print(products)

# Step 2: Soma Identification

In [ ]:
from neurd import soma_extraction_utils as sm
soma_extraction_parameters = dict()

In [ ]:
# could inspect the parameters of a certain category
from neurd import parameter_utils as paru
paru.category_param_from_module(
        module = sm,
        category = "glia",
)

In [ ]:
# overwritting decimated mesh because original was already decimated
mesh_decimated = mesh

In [ ]:
from neurd import neuron_visualizations as nviz

verbose = True

soma_products = sm.soma_indentification(
    mesh_decimated,
    verbose=verbose,
    **soma_extraction_parameters
)

In [ ]:
sm.plot_soma_products(
    mesh_decimated,
    soma_products = soma_products,
    verbose = True
)

In [ ]:
soma_products.soma_extraction_parameters = soma_extraction_parameters
soma_products

In [ ]:
products.set_stage_attrs(
    stage = "soma_identification",
    attr_dict = soma_products,
)

print(products)

# Step 2b: Saving off pipeline products

In [ ]:
from datasci_tools import system_utils as su
su.save_object(
    products,"products_up_to_soma_stage"
)

# Step 3: Decomposition

In [ ]:
from neurd import neuron

neuron_obj = neuron.Neuron(
    mesh = mesh,
    segment_id = segment_id, # don't need this explicitely if segment_id is already in products
    pipeline_products = products,
    suppress_preprocessing_print=False,
    suppress_output=False,
)

neuron_obj

In [ ]:
nviz.visualize_neuron(
    neuron_obj,
    limb_branch_dict="all"
)

# Calculate statistics

In [ ]:
decomp_products = neuron_obj.calculate_decomposition_products(
    store_in_obj = True,
)

decomp_products

In [ ]:
print(neuron_obj.pipeline_products)

# Saving off neuron

In [ ]:
vdi.save_neuron_obj(
    neuron_obj,
    verbose = True
)

# Reloading the Neuron

In [ ]:
neuron_obj_rec = vdi.load_neuron_obj(
    segment_id = segment_id,
    #mesh_decimated = mesh_decimated
)

In [ ]:
print(neuron_obj_rec.pipeline_products)

# Step 4A: Multi Soma Split Suggestions

In [ ]:
from neurd import neuron_visualizations as nviz
nviz.plot_soma_limb_concept_network(
    neuron_obj
)

In [ ]:
multi_soma_split_parameters = dict()

In [ ]:
_ = neuron_obj.calculate_multi_soma_split_suggestions(
    plot = True,
    store_in_obj = True,
    **multi_soma_split_parameters
)


In [ ]:
neuron_obj.pipeline_products.multi_soma_split_suggestions.multi_soma_split_parameters = multi_soma_split_parameters

# Step 4B: Multi Soma Split Execution

In [ ]:
neuron_list = neuron_obj.multi_soma_split_execution(
    verbose = False,
)

In [ ]:
n1 = neuron_list[0]
print(n1.pipeline_products)

# Step 5: Cell Typing

In [ ]:
from neurd import neuron_pipeline_utils as npu

In [ ]:
neuron_obj_axon = npu.cell_type_ax_dendr_stage(
    n1,
    mesh_decimated = mesh_decimated,
    plot_axon = False,
)

In [ ]:
nviz.plot_axon(
    neuron_obj_axon
)

# Autoproofreading Stage

In [ ]:
neuron_obj_proof = npu.auto_proof_stage(
    neuron_obj_axon,
    mesh_decimated = mesh_decimated,
    calculate_after_proof_stats = False,
)

In [ ]:
_ = npu.after_auto_proof_stats(
    neuron_obj_proof,
    store_in_obj = True,
)

# Saving off the autoproofread neuron

In [ ]:
vdi.save_neuron_obj_auto_proof(
    neuron_obj_proof,
)

# Plotting the final neuron

In [ ]:
from neurd import synapse_utils as syu

In [ ]:
from datasci_tools import ipyvolume_utils as ipvu
ipvu.plot_objects(
    neuron_obj_axon.mesh,
    meshes = [neuron_obj_proof.mesh],
    meshes_colors="red"
)

In [ ]:
syu.plot_synapses(neuron_obj_proof)

In [ ]:
syu.plot_synapses_error_from_neuron_obj(neuron_obj_proof)    

In [ ]:
syu.plot_synapses_valid_from_neuron_obj(neuron_obj_proof)  

In [ ]:
nviz.plot_compartments(neuron_obj_proof)

In [ ]:
from neurd import soma_extraction_utils as sm
soma_extraction_parameters = dict()

# could inspect the parameters of a certain category
from neurd import parameter_utils as paru
paru.category_param_from_module(
        module = sm,
        category = "glia",
)

# overwritting decimated mesh because original was already decimated
mesh_decimated = mesh

from neurd import neuron_visualizations as nviz

verbose = True

soma_products = sm.soma_indentification(
    mesh_decimated,
    verbose=verbose,
    **soma_extraction_parameters
)

sm.plot_soma_products(
    mesh_decimated,
    soma_products = soma_products,
    verbose = True
)

soma_products.soma_extraction_parameters = soma_extraction_parameters
soma_products

products.set_stage_attrs(
    stage = "soma_identification",
    attr_dict = soma_products,
)

print(products)

# Step 2b: Saving off pipeline products

from datasci_tools import system_utils as su
su.save_object(
    products,"products_up_to_soma_stage"
)

# Step 3: Decomposition

from neurd import neuron

neuron_obj = neuron.Neuron(
    mesh = mesh,
    segment_id = segment_id, # don't need this explicitely if segment_id is already in products
    pipeline_products = products,
    suppress_preprocessing_print=False,
    suppress_output=False,
)

neuron_obj

nviz.visualize_neuron(
    neuron_obj,
    limb_branch_dict="all"
)

# Calculate statistics

decomp_products = neuron_obj.calculate_decomposition_products(
    store_in_obj = True,
)

decomp_products

print(neuron_obj.pipeline_products)

# Saving off neuron

vdi.save_neuron_obj(
    neuron_obj,
    verbose = True
)

# Reloading the Neuron

neuron_obj_rec = vdi.load_neuron_obj(
    segment_id = segment_id,
    #mesh_decimated = mesh_decimated
)

print(neuron_obj_rec.pipeline_products)

# Step 4A: Multi Soma Split Suggestions

from neurd import neuron_visualizations as nviz
nviz.plot_soma_limb_concept_network(
    neuron_obj
)

multi_soma_split_parameters = dict()

_ = neuron_obj.calculate_multi_soma_split_suggestions(
    plot = True,
    store_in_obj = True,
    **multi_soma_split_parameters
)


neuron_obj.pipeline_products.multi_soma_split_suggestions.multi_soma_split_parameters = multi_soma_split_parameters

# Step 4B: Multi Soma Split Execution

neuron_list = neuron_obj.multi_soma_split_execution(
    verbose = False,
)

n1 = neuron_list[0]
print(n1.pipeline_products)

# Step 5: Cell Typing

from neurd import neuron_pipeline_utils as npu

neuron_obj_axon = npu.cell_type_ax_dendr_stage(
    n1,
    mesh_decimated = mesh_decimated,
    plot_axon = False,
)

nviz.plot_axon(
    neuron_obj_axon
)

# Autoproofreading Stage

neuron_obj_proof = npu.auto_proof_stage(
    neuron_obj_axon,
    mesh_decimated = mesh_decimated,
    calculate_after_proof_stats = False,
)

_ = npu.after_auto_proof_stats(
    neuron_obj_proof,
    store_in_obj = True,
)

# Saving off the autoproofread neuron

vdi.save_neuron_obj_auto_proof(
    neuron_obj_proof,
)

# Plotting the final neuron

from neurd import synapse_utils as syu

from datasci_tools import ipyvolume_utils as ipvu
ipvu.plot_objects(
    neuron_obj_axon.mesh,
    meshes = [neuron_obj_proof.mesh],
    meshes_colors="red"
)

syu.plot_synapses(neuron_obj_proof)

syu.plot_synapses_error_from_neuron_obj(neuron_obj_proof)    

syu.plot_synapses_valid_from_neuron_obj(neuron_obj_proof)  

nviz.plot_compartments(neuron_obj_proof)